In [ ]:
def get_embedding_by_date(date_ym, offset=0, edge_type='aggregate', weight_type='embed_heter_superv_recur_focal_logisticMF'):
    """
    param:
        date_ym: tuple (year, month)
        offset: month_offset, one in [3, 6]
        graph_type: one in ['latest', 'aggregate']
        weight_type: one in ['weight_log', '']
    """
    year, month = date_ym
    
    # Fetch graph embedding by customer ID
    strrrr = 'graph_embed/offset_{}/{}/{}_{}-{}.npy'.format(offset, edge_type, weight_type, year, month)
    print(strrrr)
    E_t = np.load(strrrr)
    print(E_t.shape)
    embedding = pd.DataFrame(data=E_t, 
                             index=range(E_t.shape[0]),
                             columns=['embed_{}_{}_{}'.format(edge_type, weight_type, i) for i in range(E_t.shape[1])])
    with open("GraphLaplacian/offset_3/aggregate/weight_log_nodes_int2key_{}-{}.pickle".
              format(year, month), 'rb') as f:
        int2key_t = pickle.load(f)
    embedding['customerno'] = embedding.apply(lambda x: int2key_t[x.name], axis=1)
    
    return embedding

In [ ]:
def get_feature(date_ym, pre_fix="", 
                use_feature_ctbc=True, use_feature_embed=True, use_feature_rankscore=True):
    year = date_ym[0]
    month = date_ym[1]
    from_date = pd.to_datetime("{}/{}/{}".format(month, 1, year))
    to_date = from_date + pd.DateOffset(months=1)
    print("Construct dataframe in {}.{}".format(year, month))
    
    # Get CTBC features
    if year == 2017:
        file_name = 'aml_all_{}{:02d}_y.csv'.format(str(2018)[2:], 1)
    else:
        file_name = 'aml_all_{}{:02d}_y.csv'.format(str(year)[2:], month)
    pdframe = pd.read_csv('../AMLAML/' + file_name)
    pdframe = pdframe.set_index('customerno')
    
    # Attach graph embedding onto the features
    # weight_type: [weight_none, weight_log, count_larger8000]
    embed = get_embedding_by_date(date_ym, weight_type='embed_heter_superv_recur_focal_logisticMF_embed2')
    pdframe = pdframe.join(embed.set_index('customerno'), on='customerno')
    embed_col_names = list(embed.columns)
    embed_col_names.remove('customerno')
    pdframe.loc[:, embed_col_names] = pdframe.loc[:, embed_col_names].fillna(0)
    
    # Attach ranking scores onto the features
    rscore = get_rankscore_by_date(date_ym, offset=3, edge_type='aggregate', weight_type='weight_log')
    pdframe = pdframe.join(rscore.set_index('customerno'), on='customerno')
    rscore = get_rankscore_by_date(date_ym, offset=6, edge_type='aggregate', weight_type='weight_log')
    pdframe = pdframe.join(rscore.set_index('customerno'), on='customerno')
    rscore = get_rankscore_by_date(date_ym, offset=3, edge_type='aggregate', weight_type='weight_none')
    pdframe = pdframe.join(rscore.set_index('customerno'), on='customerno')
    rscore = get_rankscore_by_date(date_ym, offset=6, edge_type='aggregate', weight_type='weight_none')
    pdframe = pdframe.join(rscore.set_index('customerno'), on='customerno')
    rscore = get_rankscore_by_date(date_ym, offset=3, edge_type='aggregate', weight_type='count_larger8000')
    pdframe = pdframe.join(rscore.set_index('customerno'), on='customerno')
    rscore = get_rankscore_by_date(date_ym, offset=6, edge_type='aggregate', weight_type='count_larger8000')
    pdframe = pdframe.join(rscore.set_index('customerno'), on='customerno')
    rscore = get_rankscore_by_date(date_ym, offset=6, edge_type='aggregate', weight_type='count_larger100000')
    pdframe = pdframe.join(rscore.set_index('customerno'), on='customerno')
    rscore_col_names = []
    for x in pdframe.columns:
        if x.startswith('pagerank') or x.startswith('divrank'):
            rscore_col_names.append(x)
    pdframe.loc[:, rscore_col_names] = pdframe.loc[:, rscore_col_names].fillna(0)
    
    # Attach label onto feature
    target_list = SARCase[(SARCase.Status_SAR == 4) & 
                          (SARCase.created_date > from_date) & 
                          (SARCase.created_date < to_date)]['customerno'].unique()
    # use ```x.name``` rather than ```x['customerno']``` since ```customerno``` has been the index not a column
    pdframe['label'] = pdframe.apply(lambda x: 1 if x.name in target_list else 0, axis=1)
    
    return pdframe

In [ ]:
from_date_ym = DateYM(2018, 3)
# to_date_ym = DateYM(2018, 1)
to_date_ym = DateYM(2019, 6)
list_date_seq = list_date_tuples(from_date_ym, to_date_ym)

AML_data = {}
embed_col_names = []
pdframe_pre = None
for date_ym in list_date_seq:
    
#     if pdframe_pre is None:
#         date_pre = DateYM(date_ym[0], date_ym[1])
#         date_pre.subtract_month(1)
#         if date_pre.is_larger_than(DateYM(2018, 1)):
#             pdframe_pre = get_feature(date_pre.export_tuple())
#         else:
#             pdframe_pre = get_feature((2018, 1))

    pdframe = get_feature(date_ym)
    
#     cols = list(pdframe.columns)
#     cols_for_diff = cols.copy()
#     cols_for_diff.remove('Customer_Type_Code')
#     cols_for_diff.remove('Customer_Category_Code')
#     pdframe = pdframe.join(pdframe[cols_for_diff] - pdframe_pre[cols_for_diff] , 
#                            rsuffix='_diff', on='customerno')
#     pdframe_pre = pdframe[cols]
        
    AML_data[date_ym] = pd.get_dummies(pdframe, columns=['Customer_Type_Code', 'Customer_Category_Code'])


# Concatenate all the datasets
# AML_dataset = pd.concat(list(AML_data.values()), axis=0)

# One-hot Encode
# AML_dataset = pd.get_dummies(AML_dataset, columns=['Customer_Type_Code', 'Customer_Category_Code'])

Construct dataframe in 2018.3
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2018-3.npy
(64712, 300)
Construct dataframe in 2018.4
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2018-4.npy
(65481, 300)
Construct dataframe in 2018.5
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2018-5.npy
(64407, 300)
Construct dataframe in 2018.6
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2018-6.npy
(66627, 300)
Construct dataframe in 2018.7
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2018-7.npy
(67397, 300)
Construct dataframe in 2018.8
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2018-8.npy
(66943, 300)
Construct dataframe in 2018.9
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2018-9.npy
(65069, 300)
Construct dataframe in 2018.10
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2018-10.npy
(65047, 300)
Construct dataframe in 2018.11
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2018-11.npy
(63888, 300)
Construct dataframe in 2018.12
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2018-12.npy
(63092, 300)
Construct dataframe in 2019.1
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2019-1.npy
(64078, 300)
Construct dataframe in 2019.2
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2019-2.npy
(59535, 300)
Construct dataframe in 2019.3
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2019-3.npy
(58637, 300)
Construct dataframe in 2019.4
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2019-4.npy
(57278, 300)
Construct dataframe in 2019.5
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2019-5.npy
(53995, 300)
Construct dataframe in 2019.6
graph_embed/offset_0/aggregate/embed_heter_superv_recur_focal_logisticMF_embed2_2019-6.npy
(55511, 300)

In [ ]:
rankscore_feat = []
for x in list(pdframe.columns):
    if (x.startswith('pagerank') or x.startswith('divrank')) and not x.endswith('_diff'):
        rankscore_feat.append(x)
print('The length of rankscore_feat = {}'.format(len(rankscore_feat)))

embed_feat = []
for x in list(pdframe.columns):
    if x.startswith('embed') and not x.endswith('_diff'):
        embed_feat.append(x)
print('The length of embed_feat = {}'.format(len(embed_feat)))

feature_feat = []
for x in list(pdframe.columns):
    if x.startswith('feature') or x.startswith('SARU'):
        feature_feat.append(x)
print('The length of feature_feat = {}'.format(len(feature_feat)))

In [ ]:
ScalarFrame = pd.DataFrame(index=rankscore_feat+embed_feat+feature_feat, columns=['max', 'min']).fillna(0)

In [ ]:
for date in AML_data.keys():
    print (date)
    tmp_AML_data = AML_data.get(date).describe()
    for n in rankscore_feat:
        if tmp_AML_data[n].max() > ScalarFrame.loc[n,'max']:
            ScalarFrame.loc[n,'max'] = tmp_AML_data[n].max()
        if tmp_AML_data[n].min() < ScalarFrame.loc[n,'min']:
            ScalarFrame.loc[n,'min'] = tmp_AML_data[n].min()
    for n in embed_feat:
        if tmp_AML_data[n].max() > ScalarFrame.loc[n,'max']:
            ScalarFrame.loc[n,'max'] = tmp_AML_data[n].max()
        if tmp_AML_data[n].min() < ScalarFrame.loc[n,'min']:
            ScalarFrame.loc[n,'min'] = tmp_AML_data[n].min()
    for n in feature_feat:
        if tmp_AML_data[n].max() > ScalarFrame.loc[n,'max']:
            ScalarFrame.loc[n,'max'] = tmp_AML_data[n].max()
        if tmp_AML_data[n].min() < ScalarFrame.loc[n,'min']:
            ScalarFrame.loc[n,'min'] = tmp_AML_data[n].min()

(2018, 3)
(2018, 4)
(2018, 5)
(2018, 6)
(2018, 7)
(2018, 8)
(2018, 9)
(2018, 10)
(2018, 11)
(2018, 12)
(2019, 1)
(2019, 2)
(2019, 3)
(2019, 4)
(2019, 5)
(2019, 6)

In [ ]:
# GRUframe_tmp
# label     = np.array([])
# announce  = np.array([])
# FILTER    = np.array([])
label    = {}
announce = {}
FILTER   = {}
GRUdict  = {}
for n in range(2, len(list_date_seq)):
    # GRUframe_tmp = None
    print (list_date_seq[n])
    GRUframe_tmp_1 = AML_data[list_date_seq[n]]
    # label = np.append(label, GRUframe_tmp_1['label'].values)
    # announce = np.append(label, GRUframe_tmp_1['announce'].values)
    # FILTER = np.append(label, GRUframe_tmp_1['FILTER'].values)
    GRUframe_tmp_1 = GRUframe_tmp_1.drop('label', axis=1)
    GRUframe_tmp_1[rankscore_feat] = (GRUframe_tmp_1[rankscore_feat] - ScalarFrame.T[rankscore_feat].min()) /(ScalarFrame.T[rankscore_feat].max() - ScalarFrame.T[rankscore_feat].min())
    GRUframe_tmp_1[embed_feat] = (GRUframe_tmp_1[embed_feat] - ScalarFrame.T[embed_feat].min()) /(ScalarFrame.T[embed_feat].max() - ScalarFrame.T[embed_feat].min())
    GRUframe_tmp_1[feature_feat] = (GRUframe_tmp_1[feature_feat] - ScalarFrame.T[feature_feat].min()) /(ScalarFrame.T[feature_feat].max() - ScalarFrame.T[feature_feat].min())
    
    
    GRUframe_tmp_2 = AML_data[list_date_seq[n-1]]
    GRUframe_tmp_2 = GRUframe_tmp_2.drop('label', axis=1)
    GRUframe_tmp_2[rankscore_feat] = (GRUframe_tmp_2[rankscore_feat] - ScalarFrame.T[rankscore_feat].min()) /(ScalarFrame.T[rankscore_feat].max() - ScalarFrame.T[rankscore_feat].min())
    GRUframe_tmp_2[embed_feat] = (GRUframe_tmp_2[embed_feat] - ScalarFrame.T[embed_feat].min()) /(ScalarFrame.T[embed_feat].max() - ScalarFrame.T[embed_feat].min())
    GRUframe_tmp_2[feature_feat] = (GRUframe_tmp_2[feature_feat] - ScalarFrame.T[feature_feat].min()) /(ScalarFrame.T[feature_feat].max() - ScalarFrame.T[feature_feat].min())

    
    GRUframe_tmp_3 = AML_data[list_date_seq[n-2]]
    GRUframe_tmp_3 = GRUframe_tmp_3.drop('label', axis=1)
    GRUframe_tmp_3[rankscore_feat] = (GRUframe_tmp_3[rankscore_feat] - ScalarFrame.T[rankscore_feat].min()) /(ScalarFrame.T[rankscore_feat].max() - ScalarFrame.T[rankscore_feat].min())
    GRUframe_tmp_3[embed_feat] = (GRUframe_tmp_3[embed_feat] - ScalarFrame.T[embed_feat].min()) /(ScalarFrame.T[embed_feat].max() - ScalarFrame.T[embed_feat].min())
    GRUframe_tmp_3[feature_feat] = (GRUframe_tmp_3[feature_feat] - ScalarFrame.T[feature_feat].min()) /(ScalarFrame.T[feature_feat].max() - ScalarFrame.T[feature_feat].min())

    
    GRUdict[list_date_seq[n]] = np.hstack([GRUframe_tmp_1.values[:,np.newaxis, :], 
                                           GRUframe_tmp_2.values[:,np.newaxis, :], 
                                           GRUframe_tmp_3.values[:,np.newaxis, :]])
    label[list_date_seq[n]]    = AML_data[list_date_seq[n]]['label'].values
    announce[list_date_seq[n]] = AML_data[list_date_seq[n]]['announce'].values
    FILTER[list_date_seq[n]]   = AML_data[list_date_seq[n]]['FILTER'].values

(2018, 5)
(2018, 6)
(2018, 7)
(2018, 8)
(2018, 9)
(2018, 10)
(2018, 11)
(2018, 12)
(2019, 1)
(2019, 2)
(2019, 3)
(2019, 4)
(2019, 5)
(2019, 6)

In [ ]:
testing_date   = [ (2018, 12),(2019, 1),(2019, 2),
                  (2019, 3),(2019, 4), (2019, 5),(2019, 6)]
training_date  = [(2018, 5),
                  (2018, 6),(2018, 7), (2018, 8),
                  (2018, 9),(2018, 10),(2018, 11)]

In [ ]:
training_data     = np.concatenate([GRUdict[date] for date in training_date])
testing_data      = np.concatenate([GRUdict[date] for date in testing_date])

training_label    = np.concatenate([label[date] for date in training_date])
testing_label     = np.concatenate([label[date] for date in testing_date])

training_announce = np.concatenate([announce[date] for date in training_date])
testing_announce  = np.concatenate([announce[date] for date in testing_date])

training_FILTER   = np.concatenate([FILTER[date] for date in training_date])
testing_FILTER    = np.concatenate([FILTER[date] for date in testing_date])

In [ ]:
GRUArray = np.vstack(list(GRUdict.values()))

np.savez('datasets/Training_data_heter.npz', training_data, training_label, training_announce, training_FILTER)
np.savez('datasets/Testing_data_heter.npz',  testing_data,  testing_label,  testing_announce,  testing_FILTER)